In [ ]:
#TODO

"""
1. predict the unlabeled data (Naive Bayes, SVM)
2. evaluate the accuracy,f1,ruc,etc?? (check with the doc later)
3. Topic Modeling
4. wordcloud(maybe,if applicable)
5. visualization(maybe, if applicable)
6. think about some other fun questions about this data set

"""

In [11]:
import pandas as pd


#settings
pd.options.display.max_rows=10

In [6]:
mapping = pd.read_csv('data/Interview_Mapping.csv')
mapping

,Judgements,Area.of.Law
0,LNIND_1988_CAL_114,To be Tested
1,LNIND_1956_CAL_163,To be Tested
2,LNIND_1976_CAL_277,To be Tested
3,LNIND_1980_CAL_52,To be Tested
4,LNIND_1955_CAL_124,To be Tested
5,LNIND_1954_CAL_3,To be Tested
6,LNIND_1951_CAL_60,To be Tested
7,LNIND_1954_CAL_46,To be Tested
8,LNIND_1976_CAL_119,To be Tested
9,LNIND_1993_DEL_120,To be Tested


In [13]:
# create labels
unlabeled = []
labeled = []
labels = []

for index,row in mapping.iterrows():
    if row['Area.of.Law'] == 'To be Tested':
        unlabeled.append(row['Judgements'])
    else: 
        labeled.append(row['Judgements'])
        labels.append(row['Area.of.Law'])
        
# how much unique area of law        
len(set(labels))

41